In [1]:
import pandas as pd
import numpy as np
import re
from itertools import chain
from math import log
from scipy.stats import chi2_contingency
from sklearn.preprocessing import LabelEncoder

In [2]:
df_origin = pd.read_excel("./無題のスプレッドシート.xlsx")

In [3]:
def split_df_isMatrix(_df):
    df_matrix = pd.DataFrame()
    df_non_matrix = pd.DataFrame()

    for column_name, item in _df.iteritems():
        if item[2] == 1:
            df_matrix = pd.concat([df_matrix, item], axis=1)
        else:
            df_non_matrix = pd.concat([df_non_matrix, item], axis=1)
            
    return df_matrix, df_non_matrix

In [4]:
# 1列目がNaNの行数を返す
def get_valid_row(_df):
    return _df.iloc[:,0].isnull().sum()

In [5]:
def edit_NUM_column(_df):
    # スタージェスの公式
    class_size = 1 + log(len(_df), 2)
    
    # ヒストグラムを作る場合の度数一覧、境界の配列のタプルを取得
    hist_result = np.histogram(_df, bins=int(class_size))
    
    # 境界で区切る
    pd.cut(_df, hist_result[1])
    
    return edit_SA_column(_df)


In [6]:
def edit_SA_column(_df):
    _df_str = _df.astype('str')
    le = LabelEncoder()
    le = le.fit(_df_str)
    
    return le.transform(_df_str)

In [7]:
def get_pairs(_df):

    columns = _df.columns

    regCompile = re.compile(r'^(Q.?\d{1,3}?)_?(.*)$')
    questionColumns = tuple(filter(regCompile.search, columns))
    questions = sorted(set(map(lambda x: regCompile.match(x).group(1), questionColumns)))

    searchTarget = tuple(map(lambda x: {x: [column for column in questionColumns if x in column]}, questions))

    pairs = []
    for i in range(len(questions)-1):
        key = questions[i]
        baseDic = next(iter([dic for dic in searchTarget if key in dic]), None)

        for item in baseDic[key]:
            for j in range(i+1, len(questions)):
                inside_key = questions[j]
                pairTuple = (tuple(dic.values())[0] for dic in searchTarget if inside_key in dic)
                pairs.extend([[item, val] for val in chain.from_iterable(pairTuple)])

    return pairs

In [20]:
def get_type_df(_df, targetType):
    
    types = ["SA", "MA", "数値", "FA"]
    
    if targetType in types:
        return _df[list(_df.iloc[1,:][_df.iloc[1,:] == targetType].index)]
    
    return None

In [9]:
validRow = get_valid_row(df_origin)
pairs = get_pairs(df_origin)
matrix, nonmatrix = split_df_isMatrix(df_origin)

In [10]:
sa_df=get_type_df(nonmatrix, "SA")
sa_df=sa_df.tail(validRow)
sa_df = sa_df.apply(edit_SA_column)
sa_df

,QStatus,Q1
9,0,0
10,2,1
11,3,2
12,4,0
13,5,1
14,6,2
15,7,0
16,8,1
17,9,2
18,1,0


In [11]:
ma_df=get_type_df(nonmatrix, "MA")
ma_df=ma_df.tail(validRow)
ma_df

,Q2_1,Q2_2,Q2_3,Q2_4,Q4_1-1,Q4_1-2,Q4_1-3,Q4_2-1,Q4_2-1.1,Q4_2-1.2,Q4_3-1,Q4_3-1.1,Q4_3-1.2
9,1,1,1,1,1,1,1,1,1,1,1,1,1
10,0,0,0,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
num_df=get_type_df(nonmatrix, "数値")
num_df=num_df.tail(validRow)
num_df=num_df.apply(edit_NUM_column)
num_df

,Q5_1
9,1
10,0
11,0
12,0
13,0
14,0
15,0
16,0
17,0
18,0


In [22]:
fa_df=get_type_df(nonmatrix, "FA")
fa_df=fa_df.tail(validRow)
fa_df

,Q3T
9,1
10,0
11,0
12,0
13,0
14,0
15,0
16,0
17,0
18,0


In [23]:
df = pd.concat([sa_df, ma_df, num_df, fa_df], axis=1)
df

,QStatus,Q1,Q2_1,Q2_2,Q2_3,Q2_4,Q4_1-1,Q4_1-2,Q4_1-3,Q4_2-1,Q4_2-1.1,Q4_2-1.2,Q4_3-1,Q4_3-1.1,Q4_3-1.2,Q5_1,Q3T
9,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
10,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,6,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,9,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [42]:
def get_chi2_contigency(_df_side, _df_head):
    
    _df = pd.crosstab(_df_side, _df_head)
    
    # correctionはイェーツの補正の有無を刺す
    returnArray = [_df_side.name, _df_head.name]
    *chi2Returns, = chi2_contingency(_df, correction=False)
#     chi2, p, dof, expected = chi2_contingency(_df, correction=False)
#     print("カイ二乗値:", chi2)
#     print("p値:", p)
#     print("自由度:", dof)
#     print("期待度数:", expected)

    returnArray.extend(chi2Returns)
    
    return returnArray

In [46]:
result = [get_chi2_contigency(df[pair[0]], df[pair[1]]) for pair in pairs]

In [47]:
result = sorted(result, key=lambda x: x[2], reverse=True)

In [48]:
result

[['Q2_1', 'Q3T', 10.0, 0.001565402258002549, 1, array([[8.1, 0.9],
         [0.9, 0.1]])],
 ['Q2_1', 'Q4_1-1', 10.0, 0.001565402258002549, 1, array([[8.1, 0.9],
         [0.9, 0.1]])],
 ['Q2_1', 'Q4_1-2', 10.0, 0.001565402258002549, 1, array([[8.1, 0.9],
         [0.9, 0.1]])],
 ['Q2_1', 'Q4_1-3', 10.0, 0.001565402258002549, 1, array([[8.1, 0.9],
         [0.9, 0.1]])],
 ['Q2_1', 'Q4_2-1', 10.0, 0.001565402258002549, 1, array([[8.1, 0.9],
         [0.9, 0.1]])],
 ['Q2_1', 'Q4_2-1.1', 10.0, 0.001565402258002549, 1, array([[8.1, 0.9],
         [0.9, 0.1]])],
 ['Q2_1', 'Q4_2-1.2', 10.0, 0.001565402258002549, 1, array([[8.1, 0.9],
         [0.9, 0.1]])],
 ['Q2_1', 'Q4_3-1', 10.0, 0.001565402258002549, 1, array([[8.1, 0.9],
         [0.9, 0.1]])],
 ['Q2_1', 'Q4_3-1.1', 10.0, 0.001565402258002549, 1, array([[8.1, 0.9],
         [0.9, 0.1]])],
 ['Q2_1', 'Q4_3-1.2', 10.0, 0.001565402258002549, 1, array([[8.1, 0.9],
         [0.9, 0.1]])],
 ['Q2_1', 'Q5_1', 10.0, 0.001565402258002549, 1, array(